# Gene expression prediction
Reference: https://www.kaggle.com/c/gene-expression-prediction

Coded by Wangjin Lee
- jinsamdol@snu.ac.kr 

#### Description
Histone modifications are playing an important role in affecting gene regulation. Nowadays, predicting gene expression from histone modification signals is a widely studied research topic.

The dataset of this competition is on "E047" (Primary T CD8+ naive cells from peripheral blood) celltype from Roadmap Epigenomics Mapping Consortium (REMC) database. For each gene, it has 100 bins with five core histone modification marks [1]. (We divide the 10,000 basepair(bp) DNA region (+/-5000bp) around the transcription start site (TSS) of each gene into bins of length 100 bp [2], and then count the reads of 100 bp in each bin. Finally, the signal of each gene has a shape of 100x5.)

The goal of this competition is to develop algorithms for accurate predicting gene expression level. High gene expression level corresponds to target label = 1, and low gene expression corresponds to target label = 0.

Thus, the inputs are 100x5 matrices and target is the probability of gene activity.

#### Data description

*In this lesson, we will only use **training set**. The set consists of 'x_train.csv' and 'y_train.csv'.*

**x_train.csv**

This file contains the feature of the genes

A single gene (Field name: **GeneId**) has 500 reads information (shape: 100 x 5 ; 100 bp and 5 histones). The value in each cell for the histone denotes the number of reads in the read cluster.

The total number of the genes : 15,485

x_train.csv file looks like this ..

| GeneId | H3K4me3 | H3K4me1 | H3K36me3 | H3K9me3 | H3K27me3 |
|---|---|---|---|---|---|
| 1 | 2 | 1 | 4 | 1 | 0 |
| 1 | 0 | 2 | 1 | 1 | 1 |
| ... |
| 2  | 1 | 0 | 1 | 0 | 0 |
| ...  |
| 15485  | 0 | 0 | 0 | 2 | 1 |

The total number of the rows: 1,548,501 (15485 x 100 + 1)

The total number of the columns: 6 (1 + 5)

**x_train.csv**

This file contains the prediction whether a gene is expressed or not.

One prediction label for one GeneId.
Prediction label is binary (1: expressed, 0: suppressed)


| GeneId | Prediction |
|---|---|
| 1 | 0 |
| 2 | 0 |
| 3 | 1 |
| ... |
| 15485 | 0 |

The total number of the rows: 15,486 (15485 + 1)

The total number of the columns: 2 (1 + 1)

## Lesson: Curate a Dataset
The cells from here until **Loading the preprocessed data** include code for loading the data from the training files and storing them into variables, and dumping them into a preprocess file.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

#### Assign variable for each the data file path

In [6]:
import os

data_path_x = os.path.abspath('./data/gene_expression/train/x_train.csv')
data_path_y = os.path.abspath('./data/gene_expression/train/y_train.csv')
print(data_path_x)

D:\206_python\tensor_board_ex\data\gene_expression\train\x_train.csv


#### Read the x_train.csv file via pandas.read_csv( ) operation

In [3]:
data_x = pd.read_csv(data_path_x, delimiter = ',', header=0)

#### Check the data description

In [4]:
data_x.describe()

,GeneId,H3K4me3,H3K4me1,H3K36me3,H3K9me3,H3K27me3
count,1.548500e+06,1.548500e+06,1.548500e+06,1.548500e+06,1.548500e+06,1.548500e+06
mean,7.743000e+03,1.628247e+00,1.533522e+00,3.368313e+00,5.099660e+00,1.118989e+00
std,4.470136e+03,2.117349e+00,1.819352e+00,4.683728e+00,1.233961e+01,1.402116e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.872000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,7.743000e+03,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00
75%,1.161400e+04,2.000000e+00,2.000000e+00,4.000000e+00,3.000000e+00,2.000000e+00
max,1.548500e+04,1.630000e+02,9.300000e+01,1.060000e+02,1.610000e+02,1.700000e+02


#### Check the some first rows in the data variable

In [5]:
data_x.head()

,GeneId,H3K4me3,H3K4me1,H3K36me3,H3K9me3,H3K27me3
0,1,2,1,4,1,0
1,1,0,2,1,1,1
2,1,0,0,4,1,1
3,1,0,2,2,0,1
4,1,2,0,0,0,0


**We will store the loaded data (x) into a list.**

The read information for a histone in a single gene is separated in 100 consecutive rows, in addition, a gene has the data for histones.

We will transform the 2-dimension data to 1-dimension.

*2d*

| GeneId | Histone1 | Histone2 | Histone3 | Histone4 | Histone5 |
|---|---|---|---|---|---|
| 1 | 1a | 1d | 1g | 1j | 1m |
| 1 | 1b | 1e | 1h | 1k | 1n |
| 1 | 1c | 1f | 1i | 1l | 1o |
| 2 | 2a | 2d | 2g | 2j | 2m |
| 2 | 2b | 2e | 2h | 2k | 2n |
| 2 | 2c | 2f | 2i | 2l | 2o |
| ... |

to *1d* 

| GeneId | Histone1-1 | Histone1-2 | Histone1-3 | Histone2-1 | ... | Histone5-3 |
|---|---|---|---|---|---|---|
| 1 | 1a | 1b | 1c | 1d | ... | 1o |
| 2 | 2a | 2b | 2c | 2d | ... | 2o |
| ... |

In [55]:
# basepair information
bp = 100
n_gene = 15485
n_histone = 5

#15485x(100*5) list. This list will store the 1d data
data_all_x = [[0]*(n_histone*bp) for i in range(n_gene)]


import itertools

col1 = []
col2 = []
col3 = []
col4 = []
col5 = []

push_idx = 0
print ('start')
for index, row in data_x.iterrows():

    if (index+1)%bp == 0:
        
        col1.append(row['H3K4me3'])
        col2.append(row['H3K4me1'])
        col3.append(row['H3K36me3'])
        col4.append(row['H3K9me3'])
        col5.append(row['H3K27me3'])
        
        #print(col1)
        #print(data_all[push_idx])
        data_all_x[push_idx] = list(itertools.chain(*[col1, col2, col3, col4, col5]))
        
        push_idx += 1
    
        col1 = []
        col2 = []
        col3 = []
        col4 = []
        col5 = []
        #print ('clear')
        if push_idx%100==0:
            print ('processed: ', push_idx)
    
    
    else:
        col1.append(row['H3K4me3'])
        col2.append(row['H3K4me1'])
        col3.append(row['H3K36me3'])
        col4.append(row['H3K9me3'])
        col5.append(row['H3K27me3'])
    

start
processed:  100
processed:  200
processed:  300
processed:  400
processed:  500
processed:  600
processed:  700
processed:  800
processed:  900
processed:  1000
processed:  1100
processed:  1200
processed:  1300
processed:  1400
processed:  1500
processed:  1600
processed:  1700
processed:  1800
processed:  1900
processed:  2000
processed:  2100
processed:  2200
processed:  2300
processed:  2400
processed:  2500
processed:  2600
processed:  2700
processed:  2800
processed:  2900
processed:  3000
processed:  3100
processed:  3200
processed:  3300
processed:  3400
processed:  3500
processed:  3600
processed:  3700
processed:  3800
processed:  3900
processed:  4000
processed:  4100
processed:  4200
processed:  4300
processed:  4400
processed:  4500
processed:  4600
processed:  4700
processed:  4800
processed:  4900
processed:  5000
processed:  5100
processed:  5200
processed:  5300
processed:  5400
processed:  5500
processed:  5600
processed:  5700
processed:  5800
processed:  5900


In [56]:
print('The first data:')
print(data_all_x[0])
print('\n')
print('The second data:')
print(data_all_x[1])

The first data:
[2, 0, 0, 0, 2, 1, 2, 1, 2, 0, 2, 0, 2, 1, 1, 0, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1, 3, 2, 3, 3, 2, 3, 1, 0, 0, 2, 0, 3, 4, 2, 0, 6, 3, 1, 3, 5, 5, 5, 2, 3, 1, 3, 0, 1, 0, 2, 1, 1, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 1, 0, 0, 2, 1, 0, 1, 4, 4, 3, 1, 1, 2, 0, 0, 3, 3, 0, 3, 1, 1, 1, 0, 0, 2, 4, 1, 1, 3, 1, 2, 4, 1, 2, 0, 2, 0, 2, 2, 1, 3, 2, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 4, 2, 1, 0, 0, 0, 3, 2, 0, 1, 2, 1, 2, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 2, 1, 4, 1, 4, 2, 0, 0, 2, 4, 4, 3, 3, 4, 5, 3, 0, 1, 2, 2, 3, 2, 3, 4, 3, 1, 1, 4, 7, 3, 4, 3, 3, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 2, 1, 2, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 2, 5, 2, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 2, 3, 2, 4, 1, 3, 9, 8, 14, 15, 12, 12, 7, 4, 5, 5, 2, 3, 2, 4

#### Read the y_train.csv file via pandas.read_csv( ) operation

The prediction is on the second column (usecols=[1])

In [7]:
data_y = pd.read_csv(data_path_y, delimiter = ',', header=0, usecols=[1])

#### Check the some first rows in the data variable

In [8]:
data_y.head()

,Prediction
0,0
1,0
2,1
3,1
4,1


In [9]:
data_y2 = data_y.values.tolist()

#### Check how the data_y2 looks like

In [10]:
print(data_y2[0:10])

[[0], [0], [1], [1], [1], [0], [1], [0], [1], [1]]


### Save the preprocessed data via pickle.dump( ) operation

In [62]:
import pickle
with open('preprocess_ge.p', 'wb') as out_file:
    pickle.dump((data_all_x, data_y2), out_file)

## Lesson: Load the dataset

#### After storing the preprocessed data into a file, we can begin at here and save our time.

In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [12]:
import pickle
def load_preprocess():
    with open('preprocess_ge.p', mode='rb') as in_file:
        return pickle.load(in_file)
(all_data_x, all_data_y) = load_preprocess()

### We need to scale (normalize) the x_data. 
Through the output from the data_x.describe() above, we checked that the data in each cell has large variance. It may lead poor prediction performance in neural network.

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_x = scaler.fit_transform(all_data_x).tolist()


In [14]:
print(scaled_x[0:2])

[[0.2797477893869934, -0.7624355412019748, -0.7573226403169175, -0.7750980346156938, 0.2726396057415219, -0.2547811711804599, 0.264078318950543, -0.27307126630112866, 0.2623155553711524, -0.7716910458734757, 0.244521952669251, -0.7877368812021848, 0.2513985051900043, -0.2566196924986161, -0.28061523738611954, -0.7779356953053257, 0.2218815061465746, -0.23921559100404186, -0.27703673766121023, -0.27810065758840774, 0.6511812492122052, -0.29572408866663147, -0.30139834475050553, -0.3031409061895812, 0.173234224176646, -0.3085504317898276, 0.6344942605038979, 0.1579002027153852, 0.6075168187927273, 0.5788682222099528, 0.13028855438080347, 0.5872091001625677, -0.3336062370307608, -0.7868838151131863, -0.7824095562175571, 0.10031950557910718, -0.7932474018709039, 0.5339440560790497, 0.9315104255622451, 0.07092520164851879, -0.8006414761856163, 1.8245077284076003, 0.5320932944600699, -0.3512686813274828, 0.5675078136174944, 1.481416908210694, 1.5115510064247073, 1.6065327977212194, 0.2204916

### One-Hot Encoding
The label in a binary or n-ary classification problem can be encoded via One-Hot Encoding method.
One-Hot encoding for binary classification problem can be seen as a method encoding label in 2-dimensional vector. 

Example: 

|Prediction |
|---|
|0|
|1|
|0|
|0|

- 1-dimensional vector: [[0], [1], [0], [0]]

- One-hot encoding (2-dimensional vector): [[1, 0], [0, 1], [1, 0], [1, 0]]

Each dimension in the 2-d vector denotes each element, thus, the *n* is larger, more zeros in the encoding vector.

In [17]:
#all_data_y_list = all_data_y.tolist()
print(all_data_y[0:5])

[[0], [0], [1], [1], [1]]


In [19]:
from sklearn.preprocessing import OneHotEncoder
data_y = OneHotEncoder(sparse=False).fit_transform(all_data_y).tolist()
print(data_y[0:5])

[[1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]


### Separation of the data → Training / Development / Testing

#### Check the X-Y pair and the number of instances

In [21]:
n_total = len(scaled_x)
print(n_total)
n_total_y = len(data_y)
assert n_total == n_total_y

15485


#### Warning!!: In the case you want to use small pieces of the data set, run this cell below, or, skip this cell

In [ ]:
'''
Using only 1/n of the whole data
'''
import math
_n=3
scaled_x = scaled_x[:math.floor(n_total/_n)]
data_y = data_y[:math.floor(n_total/_n)]

n_total = len(scaled_x)
print(n_total)
n_total_y = len(data_y)
assert n_total == n_total_y

We will divide the whole set into 7:1:2 for training, dev, and test, respectively.

In [23]:
x_data_train = scaled_x[:n_total//10*7]
y_data_train = data_y[:n_total//10*7]

x_data_dev = scaled_x[n_total//10*7:n_total//10*8]
y_data_dev = data_y[n_total//10*7:n_total//10*8]

x_data_test = scaled_x[n_total//10*8:]
y_data_test = data_y[n_total//10*8:]
print ('#train: ',' ', len(x_data_train))
print ('#dev: ',' ', len(x_data_dev))
print ('#test: ',' ', len(x_data_test))

#train:    10836
#dev:    1548
#test:    3101


## Lesson: Build the graph

#### Hyperparameters

* We will design a neural network graph consisting of one input layer, three hidden layers, and one output layer.
* In the cell below, set your hyper parameters for the node number for each layer
* Input layer (size 500) → Layer 1 (size ?) → Activation → Dropout → Layer 2 (size ?) → Activation → Dropout → Layer 3 (size ?) → Activation → Output layer (size ?) → Softmax
* Also, we can control the learning rate and *keep probability* in the dropout layer. Set your hyperparameters for the values.

In [24]:
l1=100 # the number of nodes in the layer 1
#l2=24
#l3=16
lr=0.001 #learning rate
kp=0.4 #keep probability (dropout layer)
n_iter=5000

### This is the graph structure in the TF code. 
You may use this structure and can modify the structure.
* tf.name_scope('name') will make a group in the graph. It is very useful tip making your graph compact when you see your graph in the tensorboard.
* If you want to see scalar (cost), use **tf.summary.scalar('name', variable)**
* If you want to see histogram (weights, bias), use **tf.summary.histogram('name', variable)**
* Before running the training session, type **your_summary_name=tf.summary.merge_all()**


In [25]:
'''
single layer
'''
tf.reset_default_graph()
ncol = len(scaled_x[0])
nclass = 2

with tf.name_scope('inputs'):
    
    X = tf.placeholder(tf.float32, shape=[None, ncol])
    Y = tf.placeholder(tf.float32, shape=[None, nclass])


with tf.name_scope('layer1'):
    #W1 = tf.get_variable("w1", shape=[ncol, l1], initializer=tf.contrib.layers.xavier_initializer())
    W1 = tf.Variable(tf.random_normal([ncol, l1]), name='w1')
    b1 = tf.Variable(tf.random_normal([l1]), name='b1')
    layer1 = tf.nn.relu(tf.matmul(X, W1)+b1, name='layer1')
    layer1 = tf.nn.dropout(layer1, kp)
    tf.summary.histogram("w", W1)
    tf.summary.histogram("b", b1)


with tf.name_scope('FC'):
    #FC
    #Wf = tf.get_variable("wf", shape=[l1, nclass], initializer=tf.contrib.layers.xavier_initializer())
    Wf = tf.Variable(tf.random_normal([l1, nclass]), name='wf')
    bf = tf.Variable(tf.random_normal([nclass]), name='bf')
    hypothesis = tf.nn.softmax(tf.matmul(layer1, Wf)+bf)
    tf.summary.histogram("w", Wf)
    tf.summary.histogram("b", bf)


#W = tf.Variable(tf.random_normal([ncol, 1]), name='w')
#b = tf.Variable(tf.random_normal([1]), name='b')
#hypothesis = tf.nn.softmax(tf.nn.relu(tf.matmul(X, W)+b))

with tf.name_scope('trainer'):
    #cost_2 = tf.reduce_mean(-tf.reduce_sum(Y_2*tf.log(hypothesis_2), axis=1))
    logit = tf.matmul(layer1, Wf)+bf
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels = Y))    
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)        
    
with tf.name_scope('predictor'):
    
    correct_pred = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean (tf.cast(correct_pred, tf.float32))
    
    

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()


In [27]:
'''
2 layers
'''
tf.reset_default_graph()
ncol = len(scaled_x[0])
nclass = 2

with tf.name_scope('inputs'):
    
    X = tf.placeholder(tf.float32, shape=[None, ncol])
    Y = tf.placeholder(tf.float32, shape=[None, nclass])
    #tf.summary.histogram("x2", X_2)
    #tf.summary.histogram("y2", Y_2)

with tf.name_scope('layer1'):
    W1 = tf.get_variable("w1", shape=[ncol, l1], initializer=tf.contrib.layers.xavier_initializer())
    #W1 = tf.Variable(tf.random_normal([ncol, l1]), name='w1')
    b1 = tf.Variable(tf.random_normal([l1]), name='b1')
    layer1 = tf.nn.relu(tf.matmul(X, W1)+b1, name='layer1')
    layer1 = tf.nn.dropout(layer1, kp)
    tf.summary.histogram("w", W1)
    tf.summary.histogram("b", b1)


with tf.name_scope('layer2'):
    W2 = tf.get_variable("w2", shape=[l1, l2], initializer=tf.contrib.layers.xavier_initializer())
    #W2 = tf.Variable(tf.random_normal([l1, l2]), name='w2')
    b2 = tf.Variable(tf.random_normal([l2]), name='b2')
    layer2 = tf.nn.tanh(tf.matmul(layer1, W2)+b2, name='layer2')
    layer2 = tf.nn.dropout(layer2, kp)
    tf.summary.histogram("w", W2)
    tf.summary.histogram("b", b2)

with tf.name_scope('FC'):
    #FC
    Wf = tf.get_variable("wf", shape=[l2, nclass], initializer=tf.contrib.layers.xavier_initializer())
    #Wf = tf.Variable(tf.random_normal([l2, nclass]), name='wf')
    bf = tf.Variable(tf.random_normal([nclass]), name='bf')
    hypothesis = tf.nn.softmax(tf.matmul(layer2, Wf)+bf)
    tf.summary.histogram("w", Wf)
    tf.summary.histogram("b", bf)


#W = tf.Variable(tf.random_normal([ncol, 1]), name='w')
#b = tf.Variable(tf.random_normal([1]), name='b')
#hypothesis = tf.nn.softmax(tf.nn.relu(tf.matmul(X, W)+b))

with tf.name_scope('trainer'):
    #cost_2 = tf.reduce_mean(-tf.reduce_sum(Y_2*tf.log(hypothesis_2), axis=1))
    logit = tf.matmul(layer2, Wf)+bf
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels = Y))    
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)        
    
with tf.name_scope('predictor'):
    
    correct_pred = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean (tf.cast(correct_pred, tf.float32))
    
    

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

In [46]:
'''
3 layers
'''
tf.reset_default_graph()
ncol = len(scaled_x[0])
nclass = 2

with tf.name_scope('inputs'):
    
    X = tf.placeholder(tf.float32, shape=[None, ncol])
    Y = tf.placeholder(tf.float32, shape=[None, nclass])
    #tf.summary.histogram("x2", X_2)
    #tf.summary.histogram("y2", Y_2)

with tf.name_scope('layer1'):
    W1 = tf.get_variable("w1", shape=[ncol, l1], initializer=tf.contrib.layers.xavier_initializer())
    #W1 = tf.Variable(tf.random_normal([ncol, l1]), name='w1')
    b1 = tf.Variable(tf.random_normal([l1]), name='b1')
    layer1 = tf.nn.tanh(tf.matmul(X, W1)+b1, name='layer1')
    layer1 = tf.nn.dropout(layer1, kp)
    tf.summary.histogram("w", W1)
    tf.summary.histogram("b", b1)


with tf.name_scope('layer2'):
    W2 = tf.get_variable("w2", shape=[l1, l2], initializer=tf.contrib.layers.xavier_initializer())
    #W2 = tf.Variable(tf.random_normal([l1, l2]), name='w2')
    b2 = tf.Variable(tf.random_normal([l2]), name='b2')
    layer2 = tf.nn.tanh(tf.matmul(layer1, W2)+b2, name='layer2')
    layer2 = tf.nn.dropout(layer2, kp)
    tf.summary.histogram("w", W2)
    tf.summary.histogram("b", b2)

with tf.name_scope('layer3'):
    W3 = tf.get_variable("w3", shape=[l2, l3], initializer=tf.contrib.layers.xavier_initializer())
    #W3 = tf.Variable(tf.random_normal([l2, l3]), name='w3')
    b3 = tf.Variable(tf.random_normal([l3]), name='b3')
    layer3 = tf.nn.tanh(tf.matmul(layer2, W3)+b3, name='layer3')
    layer3 = tf.nn.dropout(layer3, kp)
    tf.summary.histogram("w", W3)
    tf.summary.histogram("b", b3)

with tf.name_scope('FC'):
    #FC
    Wf = tf.get_variable("wf", shape=[l3, nclass], initializer=tf.contrib.layers.xavier_initializer())
    #Wf = tf.Variable(tf.random_normal([l3, nclass]), name='wf')
    bf = tf.Variable(tf.random_normal([nclass]), name='bf')
    hypothesis = tf.nn.softmax(tf.matmul(layer3, Wf)+bf)
    tf.summary.histogram("w", Wf)
    tf.summary.histogram("b", bf)


#W = tf.Variable(tf.random_normal([ncol, 1]), name='w')
#b = tf.Variable(tf.random_normal([1]), name='b')
#hypothesis = tf.nn.softmax(tf.nn.relu(tf.matmul(X, W)+b))

with tf.name_scope('trainer'):
    #cost_2 = tf.reduce_mean(-tf.reduce_sum(Y_2*tf.log(hypothesis_2), axis=1))
    logit = tf.matmul(layer3, Wf)+bf
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels = Y))    
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)        
    
with tf.name_scope('predictor'):
    
    correct_pred = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean (tf.cast(correct_pred, tf.float32))
    
    

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()


## Training and testing your graph.

* If you want to save tensorboard log file, type like this just after sess.run(tf.global_variables_initializer())

  **file_name = tf.summary.FileWriter('log_folder_name', sess.graph)**

* If your want to save the variables during the training, run the code

  **summary = sess.run([your_summary_name], feed_dict={....})**
  **file_name.add_summary(summary, step)**

In [27]:

sess = tf.Session()
with sess.as_default():
    
    sess.run(tf.global_variables_initializer())

    #for 1 layer
    fw2 = tf.summary.FileWriter('./tb/gene_expression/train/l1_{}-lr_{}-kp{}-init_{}'.format(l1,lr,kp,'random'), sess.graph)
    fw_val = tf.summary.FileWriter('./tb/gene_expression/validation/l1_{}-lr_{}-kp{}-init_{}'.format(l1,lr,kp,'random'))
    fw_test = tf.summary.FileWriter('./tb/gene_expression/test/l1_{}-lr_{}-kp{}-init_{}'.format(l1,lr,kp,'random'))
    
    #for 2 layers
    #fw2 = tf.summary.FileWriter('./tb/gene_expression/train/l1_{}-l2_{}-lr_{}-kp{}-init_{}'.format(l1,l2,lr,kp,'xavier_fcx'), sess.graph)
    #fw_val = tf.summary.FileWriter('./tb/gene_expression/validation/l1_{}-l2_{}-lr_{}-kp{}-init_{}'.format(l1,l2,lr,kp,'xavier_fcx'))
    #fw_test = tf.summary.FileWriter('./tb/gene_expression/test/l1_{}-l2_{}-lr_{}-kp{}-init_{}'.format(l1,l2,lr,kp,'xavier_fcx'))
    
    #for 3 layers    
    #fw2 = tf.summary.FileWriter('./tb/gene_expression/train/l1_{}-l2_{}-l3_{}-lr_{}-kp{}-init_{}_{}'.format(l1,l2,l3,lr,kp,'xavier', 'sf'), sess.graph)
    #fw_val = tf.summary.FileWriter('./tb/gene_expression/validation/l1_{}-l2_{}-l3_{}-lr_{}-kp{}-init_{}_{}'.format(l1,l2,l3,lr,kp,'xavier', 'sf'))
    #fw_test = tf.summary.FileWriter('./tb/gene_expression/test/l1_{}-l2_{}-l3_{}-lr_{}-kp{}-init_{}_{}'.format(l1,l2,l3,lr,kp,'xavier', 'sf'))

    for step in range(n_iter+1):

        
        summary, _, c = sess.run([merged, optimizer, cost], feed_dict={X:x_data_train, Y:y_data_train})
        
        if step % 200 == 0:
            summary_v, acc = sess.run([merged, accuracy], feed_dict={X:x_data_dev, Y:y_data_dev})
            print(step, '\tcost:', c, '\tacc:', acc)
            #print(summary_v)
            fw_val.add_summary(summary_v, step)
            
        fw2.add_summary(summary, step)
        
        
        
    summary_t, acc_t = sess.run([merged, accuracy], feed_dict={X:x_data_test, Y:y_data_test})    
    fw_test.add_summary(summary_t, step)
    print('final accuracy: ', acc)
    

0 	cost: 130.58849 	acc: 0.500646
200 	cost: 42.2187 	acc: 0.75323
400 	cost: 28.81076 	acc: 0.75452197
600 	cost: 20.463728 	acc: 0.7616279
800 	cost: 15.498976 	acc: 0.75645995
1000 	cost: 10.226396 	acc: 0.75323
1200 	cost: 7.710492 	acc: 0.747416
1400 	cost: 4.971384 	acc: 0.7609819
1600 	cost: 2.8837857 	acc: 0.7403101
1800 	cost: 1.2761321 	acc: 0.74418604
2000 	cost: 0.43280473 	acc: 0.7629199
2200 	cost: 0.3650502 	acc: 0.7861757
2400 	cost: 0.3609709 	acc: 0.8049096
2600 	cost: 0.36142045 	acc: 0.8068476
2800 	cost: 0.3596309 	acc: 0.81201553
3000 	cost: 0.3534212 	acc: 0.8036176
3200 	cost: 0.3525864 	acc: 0.81266147
3400 	cost: 0.3556942 	acc: 0.81459945
3600 	cost: 0.349688 	acc: 0.81072354
3800 	cost: 0.35133058 	acc: 0.81718343
4000 	cost: 0.35070935 	acc: 0.81395346
4200 	cost: 0.3473162 	acc: 0.81524545
4400 	cost: 0.34376583 	acc: 0.8217054
4600 	cost: 0.34425804 	acc: 0.81201553
4800 	cost: 0.34753305 	acc: 0.8197674
5000 	cost: 0.3393209 	acc: 0.80943155
final accura

In [26]:

sess = tf.Session()
with sess.as_default():
    
    sess.run(tf.global_variables_initializer())

    #for 1 layer
    fw2 = tf.summary.FileWriter('./tb/gene_expression/train/l1_{}-lr_{}-kp{}-init_{}'.format(l1,lr,kp,'random'), sess.graph)
    fw_val = tf.summary.FileWriter('./tb/gene_expression/validation/l1_{}-lr_{}-kp{}-init_{}'.format(l1,lr,kp,'random'))
    fw_test = tf.summary.FileWriter('./tb/gene_expression/test/l1_{}-lr_{}-kp{}-init_{}'.format(l1,lr,kp,'random'))
 
    for step in range(n_iter+1):

        
        #summary, _, c = sess.run([merged, optimizer, cost], feed_dict={X:x_data_train, Y:y_data_train})
        _, c = sess.run([optimizer, cost], feed_dict={X:x_data_train, Y:y_data_train})
        
        
        if step % 200 == 0:
            acc = sess.run([accuracy], feed_dict={X:x_data_dev, Y:y_data_dev})
            print(step, '\tcost:', c, '\tacc:', acc)
            #print(summary_v)
            fw_val.add_summary(summary_v, step)
            
        fw2.add_summary(summary, step)
        
        
        
    
    summary_t, acc_t = sess.run([merged, accuracy], feed_dict={X:x_data_test, Y:y_data_test})    
    fw_test.add_summary(summary_t, step)
    print('final accuracy: ', acc)
    

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-26-7beacd7ca707>, line 33)

## Lesson: See your graph in the Tensorboard

* Open your CMD (command) console
* Activate your working environment first.
* Let's move to your workspace folder
* Typing like this, tensorboard --logdir=./your/log/folder
* you can see the access URL looking like this (https://the_name_of_your-PC:6006)